生成复杂的图像(1000 张)


In [2]:
# === 步骤一 (v3.1): 生成 1000 张“困难”图片 ===
from PIL import Image, ImageDraw, ImageFont
import random
import os
import shutil

# --- 0. 基本参数 ---
CHAR_WIDTH = 8
CHAR_HEIGHT = 18
CHAR_SPACING = 5
START_X = 2
START_Y = 0
IMG_WIDTH = 52
IMG_HEIGHT = 18

TARGET_FOLDER = 'images_1k' # ★ 新文件夹
NUM_IMAGES = 1000           # ★ 增加数据量到 1000
# --- 1. 准备工作 ---
if os.path.exists(TARGET_FOLDER):
    shutil.rmtree(TARGET_FOLDER)
os.makedirs(TARGET_FOLDER)

# --- 2. 加载多种字体 ---
fonts = []
font_names = ["arial.ttf", "cour.ttf", "times.ttf", "verdana.ttf", "tahoma.ttf"]
font_size = 16
print("正在加载字体...")
for font_name in font_names:
    try:
        fonts.append(ImageFont.truetype(font_name, font_size))
    except IOError:
        print(f" (未找到 {font_name} 字体，跳过) ")
if not fonts:
    print("未找到任何指定字体, 使用Pillow默认字体")
    fonts.append(ImageFont.load_default())
print(f"成功加载 {len(fonts)} 种字体。")

print(f"开始在 {TARGET_FOLDER} 中生成 {NUM_IMAGES} 张“带噪声”的图片...")

# --- 3. 循环生成图片 ---
for i in range(NUM_IMAGES):
    label = ''.join([random.choice('0123456789') for _ in range(4)])
    img = Image.new('RGBA', (IMG_WIDTH, IMG_HEIGHT), color=(255, 255, 255, 255))
    for j in range(4):
        char = label[j]
        
        # 3a. 添加随机化
        font = random.choice(fonts)
        x_offset = random.randint(-2, 2)
        y_offset = random.randint(-2, 2)
        angle = random.randint(-20, 20) # 旋转

        # 3b. 绘制带旋转的字符
        char_canvas_size = (int(font_size * 1.5), int(font_size * 1.5))
        char_img = Image.new('RGBA', char_canvas_size, (255, 255, 255, 0))
        char_draw = ImageDraw.Draw(char_img)
        char_draw.text((2, 0), char, fill=(0, 0, 0, 255), font=font)
        rotated_char = char_img.rotate(angle, resample=Image.Resampling.BICUBIC, expand=False)
        paste_x = START_X + j * (CHAR_WIDTH + CHAR_SPACING) + x_offset
        paste_y = START_Y + y_offset
        img.paste(rotated_char, (paste_x, paste_y), rotated_char)

    # 4. 转换回 'L' (灰度)
    final_img = img.convert('L')
    final_draw = ImageDraw.Draw(final_img)
    # 5. 增加噪点
    for k in range(25):
        nx = random.randint(0, IMG_WIDTH - 1)
        ny = random.randint(0, IMG_HEIGHT - 1)
        final_draw.point((nx, ny), fill=0)
    # 6. 增加干扰线
    line_x1 = random.randint(0, IMG_WIDTH // 2)
    line_y1 = random.randint(0, IMG_HEIGHT)
    line_x2 = random.randint(IMG_WIDTH // 2, IMG_WIDTH)
    line_y2 = random.randint(0, IMG_HEIGHT)
    final_draw.line((line_x1, line_y1, line_x2, line_y2), fill=0, width=1)
    # 7. 保存图片
    filename = os.path.join(TARGET_FOLDER, f'{label}_{i}.png')
    final_img.save(filename)

print(f"--- {NUM_IMAGES} 张“带噪声”的图片已生成！ ---")

正在加载字体...
成功加载 5 种字体。
开始在 images_1k 中生成 1000 张“带噪声”的图片...
--- 1000 张“带噪声”的图片已生成！ ---


用投影法智能切割图片


In [4]:
# === 步骤二：处理、切割与自动打标 (按您的命令执行) ===

import os
import shutil
from PIL import Image

# ★ 修正：在这里重新声明全局变量 ★
# (使用您在“步骤一”中生成的“困难”图片文件夹)
GENERATED_IMG_FOLDER = 'images_1k'
LABELED_DATA_FOLDER = 'training_dataset'
# ★ 修正结束 ★

def get_bin_table(threshold=140):
    """
    (来自教程) 获取灰度转二值的映射table
    """
    table = []
    for i in range(256):
        if i < threshold:
            table.append(0)
        else:
            table.append(1)
    return table

def get_crop_imgs(img_binary):
    """
    (来自教程) ★ 按照您的命令，使用“固定坐标”切割
    """
    child_img_list = []
    for i in range(4):
        x = 2 + i * (8 + 5)  # 教程的“原理图”坐标
        y = 0
        child_img = img_binary.crop((x, y, x + 8, y + 18))
        child_img_list.append(child_img)
    return child_img_list

def process_and_label_images():
    """
    自动化处理、切割和打标。
    """
    print(f"\n--- 步骤二：开始处理图片并自动打标 ---")
    
    if not os.path.exists(GENERATED_IMG_FOLDER):
        print(f"错误：找不到原始图片文件夹 {GENERATED_IMG_FOLDER}。请先运行步骤一。")
        return

    if os.path.exists(LABELED_DATA_FOLDER):
        shutil.rmtree(LABELED_DATA_FOLDER)
    os.makedirs(LABELED_DATA_FOLDER)

    binary_table = get_bin_table()
    total_images = 0
    total_skipped = 0 # 记录因为切割尺寸不匹配而跳过的图片

    # --- 遍历所有生成的图片 ---
    for img_name in os.listdir(GENERATED_IMG_FOLDER):
        if not img_name.endswith('.png'):
            continue
        
        label = img_name.split('_')[0]
        img_path = os.path.join(GENERATED_IMG_FOLDER, img_name)

        # --- 1. 图像处理 (教程步骤) ---
        img = Image.open(img_path)
        img_gray = img.convert('L')
        img_binary = img_gray.point(binary_table, '1')

        # --- 2. 图像切割 (★ 调用您指定的“固定坐标”切割) ---
        child_images = get_crop_imgs(img_binary)

        # --- 3. 自动打标 ---
        if len(child_images) != len(label):
            # (这个'if'在固定切割下理论上不会触发，除非标签长度不是4)
            total_skipped += 1
            continue
            
        for i in range(len(label)):
            char_label = label[i]
            child_img = child_images[i]
            
            char_folder = os.path.join(LABELED_DATA_FOLDER, char_label)
            if not os.path.exists(char_folder):
                os.makedirs(char_folder)
                
            save_name = f"{img_name.split('.')[0]}_{i}.png"
            child_img.save(os.path.join(char_folder, save_name))
            total_images += 1

    print(f"--- 步骤二：完成！ ---")
    print(f"成功切割并打标: {total_images} 张图片")
    # print(f"因切割失败而跳过: {total_skipped} 张图片") # 这行代码在这里意义不大
    print(f"切割好的图片已存入 {LABELED_DATA_FOLDER}")

# --- 运行步骤二 ---
process_and_label_images()


--- 步骤二：开始处理图片并自动打标 ---
--- 步骤二：完成！ ---
成功切割并打标: 4000 张图片
切割好的图片已存入 training_dataset


提取分区密度特征


In [5]:
# === 步骤三：提取特征并生成 LibSVM 文件 ===

LABELED_DATA_FOLDER = 'training_dataset'
TRAIN_FILE = 'train_feature.txt'
TEST_FILE = 'test_feature.txt'

def get_feature(img):
    """
    (v4) 使用“分区密度统计”
    """
    try:
        img = img.resize((20, 20)) 
    except Exception as e:
        return None
        
    width, height = img.size
    grid_size = 4
    cell_width = width // grid_size
    cell_height = height // grid_size
    feature_vector = []
    
    for grid_y in range(grid_size):
        for grid_x in range(grid_size):
            x_start = grid_x * cell_width
            y_start = grid_y * cell_height
            x_end = x_start + cell_width
            y_end = y_start + cell_height
            
            total_pixels_in_cell = 0
            black_pixels_in_cell = 0
            for y in range(y_start, y_end):
                for x in range(x_start, x_end):
                    total_pixels_in_cell += 1
                    if img.getpixel((x, y)) == 0:
                        black_pixels_in_cell += 1
            
            if total_pixels_in_cell > 0:
                density = (black_pixels_in_cell / total_pixels_in_cell) * 100.0
            else:
                density = 0.0
            feature_vector.append(density)
            
    return feature_vector

def create_feature_files():
    print(f"\n--- 步骤三：开始提取特征并生成 LibSVM 格式文件 ---")
    
    if os.path.exists(TRAIN_FILE):
        os.remove(TRAIN_FILE)
    if os.path.exists(TEST_FILE):
        os.remove(TEST_FILE)

    f_train = open(TRAIN_FILE, 'a')
    f_test = open(TEST_FILE, 'a')
    total_train = 0
    total_test = 0

    for n in range(10):
        char_label = str(n)
        char_folder = os.path.join(LABELED_DATA_FOLDER, char_label)
        if not os.path.exists(char_folder):
            continue
            
        img_list = os.listdir(char_folder)
        split_index = int(len(img_list) * 0.8)
        
        for i, im_file in enumerate(img_list):
            im_path = os.path.join(char_folder, im_file)
            try:
                im = Image.open(im_path)
            except:
                continue
                
            im_feature = get_feature(im)
            if im_feature is None:
                continue
            
            line_parts = [char_label]
            for z_index, z_value in enumerate(im_feature):
                line_parts.append(f"{z_index + 1}:{z_value:.2f}") 
            line = ' '.join(line_parts) + '\n'
            
            if i < split_index:
                f_train.write(line)
                total_train += 1
            else:
                f_test.write(line)
                total_test += 1

    f_train.close()
    f_test.close()

    print(f"--- 步骤三：完成！ ---")
    print(f"训练集文件已保存: {TRAIN_FILE} (共 {total_train} 条数据)")
    print(f"测试集文件已保存: {TEST_FILE} (共 {total_test} 条数据)")

# --- 运行步骤三 ---
create_feature_files()


--- 步骤三：开始提取特征并生成 LibSVM 格式文件 ---
--- 步骤三：完成！ ---
训练集文件已保存: train_feature.txt (共 3197 条数据)
测试集文件已保存: test_feature.txt (共 803 条数据)


训练模型，对比核函数


In [6]:
# === 步骤四：训练模型并对比核函数 (实验核心) ===
import os, sys, importlib.util

TRAIN_FILE = 'train_feature.txt'
TEST_FILE = 'test_feature.txt'

try:
    from libsvm.svmutil import svm_read_problem, svm_train, svm_predict
    print("★★★ 成功从 libsvm.svmutil 导入！（libsvm-official）★★★")
except ModuleNotFoundError:
    try:
        from svmutil import svm_read_problem, svm_train, svm_predict
        print("★★★ 成功从顶层 svmutil 导入！（libsvm 包）★★★")
    except ModuleNotFoundError as e:
        print("\n--- 仍然导入失败 ---")
        raise

def train_and_compare_kernels():
    print(f"\n--- 步骤四：开始训练模型并对比核函数 ---")
    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print(f"错误：找不到 {TRAIN_FILE} 或 {TEST_FILE}。请先运行步骤三。")
        return

    try:
        y_train, x_train = svm_read_problem(TRAIN_FILE)
        y_test, x_test = svm_read_problem(TEST_FILE)
    except Exception as e:
        print(f"错误：读取特征文件失败。{e}")
        return

    if not y_train or not y_test:
        print("错误：训练集或测试集为空。")
        return

    kernels = {
        '线性核 (Linear, -t 0)': '-t 0 -q',
        '多项式核 (Poly, -t 1)': '-t 1 -q',
        'RBF 核 (Gaussian, -t 2)': '-t 2 -q',
        'Sigmoid 核 (-t 3)': '-t 3 -q'
    }
    results = {}

    print("\n--- ★★★ 开始执行实验 ★★★ ---")
    for kernel_name, kernel_param in kernels.items():
        print(f"\n正在训练 {kernel_name}...")
        model = svm_train(y_train, x_train, kernel_param)
        print(f"正在测试 {kernel_name}...")
        p_labels, p_acc, p_vals = svm_predict(y_test, x_test, model)
        accuracy = p_acc[0]
        results[kernel_name] = accuracy
        print(f"--- {kernel_name} 的准确率 (Accuracy): {accuracy:.2f}% ---")

    print("\n\n--- ★★★ 最终实验对比结果 ★★★ ---")
    for kernel_name, accuracy in results.items():
        print(f"{kernel_name.ljust(25)}: {accuracy:.2f}%")
    print("---------------------------------")
    print("\n--- 实验任务全部完成！ ---")

# 运行
train_and_compare_kernels()

★★★ 成功从 libsvm.svmutil 导入！（libsvm-official）★★★

--- 步骤四：开始训练模型并对比核函数 ---

--- ★★★ 开始执行实验 ★★★ ---

正在训练 线性核 (Linear, -t 0)...
正在测试 线性核 (Linear, -t 0)...
Accuracy = 27.3973% (220/803) (classification)
--- 线性核 (Linear, -t 0) 的准确率 (Accuracy): 27.40% ---

正在训练 多项式核 (Poly, -t 1)...
正在测试 多项式核 (Poly, -t 1)...
Accuracy = 43.5866% (350/803) (classification)
--- 多项式核 (Poly, -t 1) 的准确率 (Accuracy): 43.59% ---

正在训练 RBF 核 (Gaussian, -t 2)...
正在测试 RBF 核 (Gaussian, -t 2)...
Accuracy = 10.7098% (86/803) (classification)
--- RBF 核 (Gaussian, -t 2) 的准确率 (Accuracy): 10.71% ---

正在训练 Sigmoid 核 (-t 3)...
正在测试 Sigmoid 核 (-t 3)...
Accuracy = 9.83811% (79/803) (classification)
--- Sigmoid 核 (-t 3) 的准确率 (Accuracy): 9.84% ---


--- ★★★ 最终实验对比结果 ★★★ ---
线性核 (Linear, -t 0)       : 27.40%
多项式核 (Poly, -t 1)        : 43.59%
RBF 核 (Gaussian, -t 2)   : 10.71%
Sigmoid 核 (-t 3)         : 9.84%
---------------------------------

--- 实验任务全部完成！ ---
